In [1]:
import header
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Javascript
from tkinter import Tk, filedialog
from src.utils import j_loads, j_dumps, j_loads_ns
from src.utils import pprint

# Function to create input widgets based on the JSON structure
def create_widgets(data):
    promotion_name = widgets.Text(value=data.promotion_name, description='Promotion Name:')
    title = widgets.Text(value=data.title, description='Title:')
    category_widgets = {}
    
    for category, details in data.categories.items():
        category_title = widgets.Text(value=details.title, description=f'{category} Title:')
        category_description = widgets.Text(value=details.description, description=f'{category} Description:')
        tags = widgets.Text(value=', '.join(details.tags), description=f'{category} Tags:')
        category_widgets[category] = (category_title, category_description, tags)
    
    return promotion_name, title, category_widgets

# Function to update JSON structure based on input widget values
def update_json(change, data, file_path):
    data.promotion_name = promotion_name.value
    data.title = title.value
    for category, (category_title, category_description, tags) in category_widgets.items():
        data.categories[category].title = category_title.value
        data.categories[category].description = category_description.value
        data.categories[category].tags = [tag.strip() for tag in tags.value.split(',')]
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(j_dumps(data, indent=4, ensure_ascii=False))

# Function to add a new category
def add_category(b):
    category_name = new_category_name.value.strip()
    if category_name and category_name not in data.categories:
        data.categories[category_name] = j_loads_ns('{"title": "", "description": "", "tags": []}')
        category_title = widgets.Text(value="", description=f'{category_name} Title:')
        category_description = widgets.Text(value="", description=f'{category_name} Description:')
        tags = widgets.Text(value="", description=f'{category_name} Tags:')
        category_widgets[category_name] = (category_title, category_description, tags)
        
        category_title.observe(on_change, names='value')
        category_description.observe(on_change, names='value')
        tags.observe(on_change, names='value')
        
        display(category_title, category_description, tags)

# Function to select a JSON file
def select_file(b):
    display(Javascript('alert("Changes saved to ' + current_file_path + '")'))
    root = Tk()
    root.withdraw()  # Hide the root window
    file_path = filedialog.askopenfilename(filetypes=[("JSON files", "*.json")])
    root.destroy()
    if file_path:
        global data, current_file_path
        current_file_path = file_path
        data = j_loads_ns(file_path)
        clear_widgets()
        create_and_display_widgets(data, file_path)

# Function to clear existing widgets
def clear_widgets():
    display_area.clear_output()

# Function to create and display widgets
def create_and_display_widgets(data, file_path):
    global promotion_name, title, category_widgets
    promotion_name, title, category_widgets = create_widgets(data)

    promotion_name.observe(on_change, names='value')
    title.observe(on_change, names='value')
    for category, widgets_tuple in category_widgets.items():
        for widget in widgets_tuple:
            widget.observe(on_change, names='value')

    with display_area:
        display(promotion_name, title)
        for widgets_tuple in category_widgets.values():
            display(*widgets_tuple)
        display(new_category_name, add_category_button)
        display(save_button)

# Attach the update function to the input widgets
def on_change(change):
    update_json(change, data, current_file_path)

# Load JSON file button
load_button = widgets.Button(description="Load JSON File")
load_button.on_click(select_file)

# Input field and button to add a new category
new_category_name = widgets.Text(value="", description="New Category Name:")
add_category_button = widgets.Button(description="Add Category")
add_category_button.on_click(add_category)

# Save button to manually save the changes
save_button = widgets.Button(description="Save Changes")
def on_save_button_clicked(b):
    update_json(None, data, current_file_path)
    print("Changes saved to", current_file_path)

save_button.on_click(on_save_button_clicked)

# Display the load button
display(load_button)

# Output area to display widgets
display_area = widgets.Output()
display(display_area)

Button(description='Load JSON File', style=ButtonStyle())

Output()

In [ ]:
pprint(data)